In [1]:
import pandas as pd
import psycopg2
import GeoHeyML as gml

In [2]:
conn = psycopg2.connect(database='maplead',host='192.168.31.15',user='dev',port=10294,password='zxcvbnm@1')

In [3]:
sql1 = "select * from tmp.lk_stores_train"
sql2 = "select * from tmp.lk_stores_predict"

In [4]:
df_ori = pd.read_sql(sql1, conn)

### 读取训练数据，自动特征工程与自动调参

In [5]:
df_x = df_ori.drop(['avg_order','base_id','city'], axis=1)
df_y = df_ori.avg_order

In [6]:
processer = gml.GeoHeyMLPreProcess() #初始化预处理
df_x_FE, df_y_FE = processer.autoFE(df_x,df_y)#自动化特征工程
model_params = processer.autoTP()#自动化调参

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] n_estimators=20 .................................................
[CV] .................... n_estimators=20, score=-0.260, total=   0.1s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[CV] n_estimators=20 .................................................
[CV] .................... n_estimators=20, score=-0.211, total=   0.1s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[CV] n_estimators=20 .................................................
[CV] .................... n_estimators=20, score=-0.243, total=   0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[CV] n_estimators=20 .................................................
[CV] .................... n_estimators=20, score=-0.198, total=   0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapse

[CV] ................... n_estimators=393, score=-0.222, total=   0.4s
[Parallel(n_jobs=1)]: Done  37 out of  37 | elapsed:    9.7s remaining:    0.0s
[CV] n_estimators=393 ................................................
[CV] ................... n_estimators=393, score=-0.240, total=   0.3s
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:   10.0s remaining:    0.0s
[CV] n_estimators=393 ................................................
[CV] ................... n_estimators=393, score=-0.222, total=   0.3s
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:   10.4s remaining:    0.0s
[CV] n_estimators=393 ................................................
[CV] ................... n_estimators=393, score=-0.224, total=   0.4s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:   10.8s remaining:    0.0s
[CV] n_estimators=446 ................................................
[CV] ................... n_estimators=446, score=-0.259, total=   0.4s
[Parallel(n_jobs=1)]: Done  41 out of  41

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] learning_rate=0.04 ..............................................
[CV] ................. learning_rate=0.04, score=-0.254, total=   0.1s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[CV] learning_rate=0.04 ..............................................
[CV] ................. learning_rate=0.04, score=-0.210, total=   0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[CV] learning_rate=0.04 ..............................................
[CV] ................. learning_rate=0.04, score=-0.238, total=   0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[CV] learning_rate=0.04 ..............................................
[CV] ................. learning_rate=0.04, score=-0.197, total=   0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed

### 读取待预测数据，进行相同的特征工程

In [7]:
df_pred_ori = pd.read_sql(sql2, conn)
df_pred = df_pred_ori.drop(['base_id','city'],axis=1)
df_pred_FE = gml.GeoHeyMLPreProcess().autoFE(df_pred)#待预测样本自动特征工程

### 模型训练

In [8]:
model = gml.GeoHeyMLRegressionModel(model='lightgbm')#初始化回归模型
model.train(model_params, df_x_FE, df_y_FE)#训练模型

/Users/geohey/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	training's rmse: 0.236767	valid_1's rmse: 0.24048
Training until validation scores don't improve for 20 rounds
[2]	training's rmse: 0.234537	valid_1's rmse: 0.239701
[3]	training's rmse: 0.232401	valid_1's rmse: 0.238911
[4]	training's rmse: 0.230403	valid_1's rmse: 0.238455
[5]	training's rmse: 0.22841	valid_1's rmse: 0.237465
[6]	training's rmse: 0.226508	valid_1's rmse: 0.23668
[7]	training's rmse: 0.224676	valid_1's rmse: 0.235875
[8]	training's rmse: 0.222934	valid_1's rmse: 0.235314
[9]	training's rmse: 0.221256	valid_1's rmse: 0.235011
[10]	training's rmse: 0.219707	valid_1's rmse: 0.234592
[11]	training's rmse: 0.217906	valid_1's rmse: 0.234423
[12]	training's rmse: 0.21612	valid_1's rmse: 0.233744
[13]	training's rmse: 0.214433	valid_1's rmse: 0.233221
[14]	training's rmse: 0.212832	valid_1's rmse: 0.233367
[15]	training's rmse: 0.211276	valid_1's rmse: 0.232797
[16]	training's rmse: 0.209817	valid_1's rmse: 0.231909
[17]	training's rmse: 0.208386	valid_1's rmse: 0.231383


### 输出特征重要性

In [14]:
fea_importance_dict = model.feature_importance()

In [19]:
fea_importance_dict

{'features': Index(['competitor_total', 'competitor_brand', 'competitor_price',
        'self_compete_total', 'mall_total', 'supermarket_total',
        'convinence_store_total', 'lifestyle_service_total', 'catering_price',
        'restaurant_total', 'shopping_price', 'shopping_total',
        'industry_price', 'industry_total', 'bus_total', 'bus_station_total',
        'subway_total', 'subway_station_total', 'pop_total', 'worker_total',
        'resident_total', 'resident_home_total', 'resident_price',
        'resident_price_median', 'office_total', 'office_price',
        'office_price_median', 'office_price_rk', 'real_estate_rk',
        'starbucks_total'],
       dtype='object'),
 'values': array([0.1346989 , 0.68701581, 1.0265642 , 2.45135819, 0.3646985 ,
        0.0456358 , 0.861208  , 1.76780579, 1.61389981, 7.32333492,
        2.7111804 , 0.5178795 , 0.2435419 , 0.9623223 , 0.8840205 ,
        1.43250741, 0.4178571 , 0.2638313 , 1.785969  , 6.10043041,
        3.1097996 , 0.8